In [2]:
import torch
import torch.nn as nn
nn.Linear(1, 2)(torch.tensor([0.8]))

tensor([0.8741, 1.0981], grad_fn=<AddBackward0>)

In [8]:
gt = torch.randn([16, 20, 80, 80])
std = torch.randn([16, 1, 1, 1])
std_expand = std.expand(-1, -1, h, w)
gt = torch.cat([gt, std_expand], dim=1)

In [9]:
gt.size()

torch.Size([16, 21, 80, 80])

In [1]:
import sys
sys.path.append('/data_young/super_resolution2/SRFlow/code/')
#import models.modules.StdConditional.StdConditionalLayer

In [87]:
import torch
from torch import nn as nn

from models.modules import thops
from utils.util import opt_get


class StdConditionalLayer(nn.Module):
    def __init__(self, in_channels):
        super().__init__()
        self.scaleLayer = nn.Linear(1, in_channels)
        self.shiftLayer = nn.Linear(1, in_channels)

    def forward(self, input: torch.Tensor, logdet=None, reverse=False, std=None):
        if not reverse:
            z = input
            b = z.size(0)
            # Std Conditional
            scaleFt, shiftFt = torch.sigmoid(self.scaleLayer(std.view(-1,1))).view(b, -1, 1, 1), self.shiftLayer(std.view(-1,1)).view(b, -1, 1, 1)
            z = z + shiftFt
            z = z * scaleFt
            logdet = logdet + self.get_logdet(scaleFt)
            
            output = z
        else:
            z = input
            b = z.size(0)
            # Std Conditional
            scaleFt, shiftFt = torch.sigmoid(self.scaleLayer(std.view(-1,1))).view(b, -1, 1, 1), self.shiftLayer(std.view(-1,1)).view(b, -1, 1, 1)
            z = z / scale
            z = z - shift
            logdet = logdet - self.get_logdet(scaleFt)

            output = z
        return output, logdet

    def get_logdet(self, scale):
        return thops.sum(torch.log(scale), dim=[1])


In [88]:
stdL = StdConditionalLayer(12)

In [89]:
input = torch.ones([16, 12, 80, 80])
std=torch.randn([16, 1, 1, 1])
stdL(input, std=std)

torch.Size([16, 12, 80, 80])
torch.Size([16, 12, 1, 1])
tensor([[[ -9.1392]],

        [[ -8.9860]],

        [[ -9.4006]],

        [[-10.5550]],

        [[ -9.0152]],

        [[ -9.1680]],

        [[ -9.2981]],

        [[ -8.9835]],

        [[ -8.9842]],

        [[ -9.1849]],

        [[ -9.0086]],

        [[-10.8575]],

        [[-10.6167]],

        [[ -9.1143]],

        [[ -9.8286]],

        [[ -9.0117]]], grad_fn=<SqueezeBackward3>)


TypeError: unsupported operand type(s) for +: 'NoneType' and 'Tensor'

In [71]:

class _ActNorm(nn.Module):
    """
    Activation Normalization
    Initialize the bias and scale with a given minibatch,
    so that the output per-channel have zero mean and unit variance for that.
    After initialization, `bias` and `logs` will be trained as parameters.
    """

    def __init__(self, num_features, scale=1.):
        super().__init__()
        # register mean and scale
        size = [1, num_features, 1, 1]
        self.register_parameter("bias", nn.Parameter(torch.zeros(*size)))
        self.register_parameter("logs", nn.Parameter(torch.zeros(*size)))
        self.num_features = num_features
        self.scale = float(scale)
        self.inited = False

    def _check_input_dim(self, input):
        return NotImplemented

    def initialize_parameters(self, input):
        self._check_input_dim(input)
        if not self.training:
            return
        if (self.bias != 0).any():
            self.inited = True
            return
        assert input.device == self.bias.device, (input.device, self.bias.device)
        with torch.no_grad():
            bias = thops.mean(input.clone(), dim=[0, 2, 3], keepdim=True) * -1.0
            vars = thops.mean((input.clone() + bias) ** 2, dim=[0, 2, 3], keepdim=True)
            logs = torch.log(self.scale / (torch.sqrt(vars) + 1e-6))
            self.bias.data.copy_(bias.data)
            self.logs.data.copy_(logs.data)
            self.inited = True

    def _center(self, input, reverse=False, offset=None):
        bias = self.bias

        if offset is not None:
            bias = bias + offset

        if not reverse:
            return input + bias
        else:
            return input - bias

    def _scale(self, input, logdet=None, reverse=False, offset=None):
        logs = self.logs

        if offset is not None:
            logs = logs + offset

        if not reverse:
            input = input * torch.exp(logs) # should have shape batchsize, n_channels, 1, 1
            # input = input * torch.exp(logs+logs_offset)
        else:
            input = input * torch.exp(-logs)
        if logdet is not None:
            """
            logs is log_std of `mean of channels`
            so we need to multiply pixels
            """
            dlogdet = thops.sum(logs) * thops.pixels(input)
            if reverse:
                dlogdet *= -1
            logdet = logdet + dlogdet
        return input, logdet

    def forward(self, input, logdet=None, reverse=False, offset_mask=None, logs_offset=None, bias_offset=None):
        if not self.inited:
            self.initialize_parameters(input)
        self._check_input_dim(input)

        if offset_mask is not None:
            logs_offset *= offset_mask
            bias_offset *= offset_mask
        # no need to permute dims as old version
        if not reverse:
            # center and scale

            # self.input = input
            input = self._center(input, reverse, bias_offset)
            input, logdet = self._scale(input, logdet, reverse, logs_offset)
        else:
            # scale and center
            input, logdet = self._scale(input, logdet, reverse, logs_offset)
            input = self._center(input, reverse, bias_offset)
        return input, logdet

In [10]:
def sum(tensor, dim=None, keepdim=False):
    if dim is None:
        # sum up all dim
        return torch.sum(tensor)
    else:
        if isinstance(dim, int):
            dim = [dim]
        dim = sorted(dim)
        for d in dim:
            tensor = tensor.sum(dim=d, keepdim=True)
        if not keepdim:
            for i, d in enumerate(dim):
                tensor.squeeze_(d-i)
        return tensor


def mean(tensor, dim=None, keepdim=False):
    if dim is None:
        # mean all dim
        return torch.mean(tensor)
    else:
        if isinstance(dim, int):
            dim = [dim]
        dim = sorted(dim)
        for d in dim:
            tensor = tensor.mean(dim=d, keepdim=True)
        if not keepdim:
            for i, d in enumerate(dim):
                tensor.squeeze_(d-i)
        return tensor


def split_feature(tensor, type="split"):
    """
    type = ["split", "cross"]
    """
    C = tensor.size(1)
    if type == "split":
        return tensor[:, :C // 2, ...], tensor[:, C // 2:, ...]
    elif type == "cross":
        return tensor[:, 0::2, ...], tensor[:, 1::2, ...]


def cat_feature(tensor_a, tensor_b):
    return torch.cat((tensor_a, tensor_b), dim=1)


def pixels(tensor):
    return int(tensor.size(2) * tensor.size(3))

In [23]:
input = torch.ones([1, 3, 1, 1])

In [39]:
scale = torch.tensor([[0.4, 0.5], [0.8, 0.5]])
vars = mean((input.clone()) ** 2, dim=[0, 2, 3], keepdim=True)
logs = torch.log(scale / (torch.sqrt(vars) + 1e-6))

In [40]:
logs

tensor([[[[-0.9163, -0.6931],
          [-0.2231, -0.6931]],

         [[-0.9163, -0.6931],
          [-0.2231, -0.6931]],

         [[-0.9163, -0.6931],
          [-0.2231, -0.6931]]]])

In [41]:
sum(logs) * pixels(input)

tensor(-7.5772)

In [42]:
sum(torch.log(scale), dim=[1])

tensor([-1.6094, -0.9163])

In [43]:
torch.log(scale)

tensor([[-0.9163, -0.6931],
        [-0.2231, -0.6931]])

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

from models.modules.FlowActNorms import ActNorm2d
from models.modules import thops


class Conv2d(nn.Conv2d):
    pad_dict = {
        "same": lambda kernel, stride: [((k - 1) * s + 1) // 2 for k, s in zip(kernel, stride)],
        "valid": lambda kernel, stride: [0 for _ in kernel]
    }

    @staticmethod
    def get_padding(padding, kernel_size, stride):
        # make paddding
        if isinstance(padding, str):
            if isinstance(kernel_size, int):
                kernel_size = [kernel_size, kernel_size]
            if isinstance(stride, int):
                stride = [stride, stride]
            padding = padding.lower()
            try:
                padding = Conv2d.pad_dict[padding](kernel_size, stride)
            except KeyError:
                raise ValueError("{} is not supported".format(padding))
        return padding

    def __init__(self, in_channels, out_channels,
                 kernel_size=[3, 3], stride=[1, 1],
                 padding="same", do_actnorm=True, weight_std=0.05):
        padding = Conv2d.get_padding(padding, kernel_size, stride)
        super().__init__(in_channels, out_channels, kernel_size, stride,
                         padding, bias=(not do_actnorm))
        # init weight with std
        self.weight.data.normal_(mean=0.0, std=weight_std)
        if not do_actnorm:
            self.bias.data.zero_()
        else:
            self.actnorm = ActNorm2d(out_channels)
        self.do_actnorm = do_actnorm

    def forward(self, input):
        x = super().forward(input)
        if self.do_actnorm:
            x, _ = self.actnorm(x)
        return x

In [64]:
class Linear(nn.Linear):
    def __init__(self, in_features, out_features,
                  do_actnorm=True, weight_std=0.05):
        super().__init__(in_features, out_features, bias=(not do_actnorm))
        # init weight with std
        self.weight.data.normal_(mean=0.0, std=weight_std)
        if not do_actnorm:
            self.bias.data.zero_()
        else:
            self.actnorm = ActNorm2d(out_features)
        self.do_actnorm = do_actnorm

    def forward(self, input):
        b, c = input.size(0), input.size(1)
        x = super().forward(input.view(b, c))  
        b, c = x.size(0), x.size(1)
        print(x.size())
        if self.do_actnorm and len(x.size())==2:
            print(x)
            y = x.view(b, c, 1, 1)
            x, _ = self.actnorm(y)
        x = x.view(b, c, 1, 1)
        return x

In [65]:
class LinearZeros(nn.Linear):
    def __init__(self, in_features, out_features,
                  do_actnorm=True, weight_std=0.05, logscale_factor=3):
        super().__init__(in_features, out_features, bias=True)
        # logscale_factor
        self.logscale_factor = logscale_factor
        self.register_parameter("logs", nn.Parameter(torch.zeros(out_features)))
        # init
        self.weight.data.zero_()
        self.bias.data.zero_()

    def forward(self, input):
        output = super().forward(input)
        return output * torch.exp(self.logs * self.logscale_factor)

In [66]:
input = torch.randn([1,1])
Linear(1, 20)(input).size()

torch.Size([1, 20])
tensor([[ 0.0329, -0.0244,  0.0493,  0.0084,  0.0406, -0.0476, -0.0140,  0.0439,
         -0.0613, -0.1106, -0.0383,  0.0692, -0.0007, -0.0375,  0.0357, -0.0081,
         -0.0023, -0.0241, -0.0656, -0.0419]], grad_fn=<MmBackward>)


torch.Size([1, 20, 1, 1])

In [50]:
LinearZeros(1,20)(input).size()

torch.Size([1, 20])